In [1]:
import json
import pandas as pd
from datasets import load_from_disk


data = json.load(open('../data/raw/wikipedia_documents.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])

mrc = pd.concat([train_df, valid_df])

wiki.shape, mrc.shape

/opt/conda/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


((60613, 8), (4192, 7))

In [54]:
mrc.head(1)

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42


In [9]:
from sentence_transformers import SentenceTransformer

model_name = "nlpai-lab/KoE5"
model = SentenceTransformer(
    model_name_or_path=model_name, 
    )
tokenizer = model.tokenizer
max_seq_length = min(model.max_seq_length, model[0].auto_model.config.max_position_embeddings)  # 최대 시퀀스 길이

In [3]:
import re
from tqdm import tqdm
tqdm.pandas()  # tqdm을 pandas와 함께 사용 가능하게 설정

def check_line_breaks(text):
    print(repr(text))
    
def sentence_split(text):
    # 간단한 문장 단위 분리 함수 (구두점 기준으로 문장을 분리)
    sentences = re.split(r'(?<=[.!?])\s+|\n+|\\n+|\xa0+|。', text)
    
    return sentences

def check_token_length(wiki, tokenizer, max_seq_length):
    """
    문서를 토큰화하고, 각 문서의 가장 긴 문장 토큰 길이를 계산하여,
    max_seq_length를 초과하는 문서를 필터링하고 정렬된 결과를 반환하는 함수.

    Parameters:
    wiki (pd.DataFrame): 텍스트가 포함된 데이터프레임
    tokenizer: 문장을 토큰화하는 tokenizer 객체
    max_seq_length (int): 최대 시퀀스 길이

    Returns:
    pd.DataFrame: max_seq_length를 초과하는 긴 문장이 포함된 문서들
    """
    wiki = wiki.copy()
    wiki['context_sentences'] = wiki['text'].progress_apply(lambda x: sentence_split(x))
    wiki['context_sentences_token_len_max'] = wiki['context_sentences'].progress_apply(lambda x: max([len(tokenizer.tokenize(sentence)) for sentence in x]))
    print(wiki['context_sentences_token_len_max'].describe())
    wiki_long = wiki[wiki['context_sentences_token_len_max'] > max_seq_length]
    wiki_long = wiki_long.sort_values(by='context_sentences_token_len_max', ascending=False)
    
    return wiki_long

def see_long_sentence(wiki_long, tokenizer, idx):
    sentences = []
    print(wiki_long['context_sentences_token_len_max'].iloc[idx])
    for sentence in wiki_long['context_sentences'].iloc[idx]:
        if len(tokenizer.tokenize(sentence)) > max_seq_length:
            check_line_breaks(sentence)
            print('*'*100)
            print(tokenizer.tokenize(sentence))
            print('*'*100)
            print(sentence)
            sentences.append(sentence)
    return sentences
    
    # print(wiki_long['context_sentences'].iloc[idx])

In [3]:
import json
import pandas as pd
from tqdm import tqdm
import re
from sentence_transformers import SentenceTransformer

def split_wiki_for_model(model_name="nlpai-lab/KoE5", path='/data/preprocessed/wikipedia_documents_processed.json', return_path='/data/preprocessed/wikipedia_documents_splitted.json'):
    def sentence_split(text):
        # 간단한 문장 단위 분리 함수 (구두점 기준으로 문장을 분리)
        sentences = re.split(r'(?<=[.!?])\s+|\n+|\\n+|\xa0+|。', text)
        return sentences

    def process_sentences(df, tokenizer, max_seq_length):
        print(
            """
            *******************************************************************
            DataFrame의 각 row에 있는 문장들을 토큰화하고, 주어진 최대 시퀀스 길이에 맞게 문장을 분할하여 처리하는 함수입니다.

            각 row에 대해:
            - 각 문장을 토큰화합니다.
            - 토큰화된 문장들을 합쳐서 최대 시퀀스 길이를 넘기지 않는 범위에서 처리합니다.
            - 토큰 길이가 max_seq_length를 넘으면, 새로운 row를 생성하고 합쳐진 문장들을 저장한 후 다시 길이를 초기화합니다.
            - 모든 문장을 처리할 때까지 반복합니다.

            반환:
            - 처리된 결과가 담긴 새로운 DataFrame을 반환합니다.
            *******************************************************************
        """
        )
        new_rows = []
        
        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Chunking wiki documents to fit model input size"):
            sentences = row['context_sentences']
            token_length_sum = 0
            current_text = []
            
            for sentence in sentences:
                token_length = len(tokenizer.tokenize(sentence))
                
                if token_length_sum + token_length > max_seq_length:
                    new_row = row.copy()
                    new_row['text_processed'] = " ".join(current_text) # text_processed 대신 text로 나중에 바꾸기
                    new_rows.append(new_row)
                    
                    token_length_sum = 0
                    current_text = []
                
                token_length_sum += token_length
                current_text.append(sentence)
            
            if current_text:
                new_row = row.copy()
                new_row['text_processed'] = " ".join(current_text)
                new_rows.append(new_row)
        
        return pd.DataFrame(new_rows)

    model = SentenceTransformer(
        model_name_or_path=model_name, 
        )
    tokenizer = model.tokenizer
    max_seq_length = min(model.max_seq_length, model[0].auto_model.config.max_position_embeddings)  # 최대 시퀀스 길이

    data_processed = json.load(open(path))
    wiki_p = pd.DataFrame(data_processed).T
    print(wiki_p.shape)
    wiki_p.head()

    tqdm.pandas()

    wiki_p['context_sentences'] = wiki_p['text'].progress_apply(lambda x: sentence_split(x))
    wiki_p.head()

    # Process the DataFrame
    processed_df = process_sentences(df=wiki_p, tokenizer=tokenizer, max_seq_length=max_seq_length)
    print(processed_df.shape)
    processed_df['text'] = processed_df['text_processed']
    processed_df = processed_df.drop(columns=['context_sentences', 'text_processed'])
    processed_df = processed_df.reset_index(drop=True)
    data = processed_df.to_dict(orient='index')

    with open(return_path, 'w') as f:
        json.dump(data, f)
        
wiki_split(model_name="nlpai-lab/KoE5", 
           path='../data/preprocessed/wikipedia_documents_processed.json', 
           return_path='../data/preprocessed/wikipedia_documents_splitted_for_retrieval.json')


(56808, 8)


Chunking wiki documents to fit model input size: 100%|██████████| 56808/56808 [02:37<00:00, 361.30it/s]


(75112, 10)


In [26]:
data = processed_df.to_dict(orient='index')
data

{0: {'text': '나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. # 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. # 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
  'corpus_source': '위키피디아',
  'url': 'TODO',
  'domain': None,
  'title': '나라 목록',
  'author': None,
  'html': None,
  'document_id': 0},
 1: {'text': '나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다. * (a) 영속적인 국민 * (b) 일정한 영토 * (c) 정부 * (d) 타국과의 관계 참여 자격. 특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다. 이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 

In [19]:
data_processed = json.load(open('../data/preprocessed/wikipedia_documents_processed.json'))
wiki = pd.DataFrame(data_processed).T
print(wiki.columns)

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id'],
      dtype='object')


In [17]:
wiki_p.columns

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id', 'context_sentences'],
      dtype='object')

In [18]:
processed_df.columns

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id', 'context_sentences', 'text_processed',
       'text_processed_token_len'],
      dtype='object')

In [21]:
processed_df.head(10)

,text,corpus_source,url,domain,title,author,html,document_id,context_sentences,text_processed,text_processed_token_len
0,"나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인...",위키피디아,TODO,None,나라 목록,None,None,0,"[나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승...","나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인...",228
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,504
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,남극 조약에 따라서 규제되는 이 권리는 어떤 나라에서도 승인을 받지 않았다. * E...,482
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,"기사단의 단원은 모두 다른 나라의 국민이기 때문에, 사실상 이들은 모두 자신의 국가...",208
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,497
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 ...,504
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,"이 전시는 백남준이 자신의 즉흥음악 또는 무음악의 발상에 기초한 실제 퍼포먼스, 그...",479
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,특히 1967년 음악에 성적인 코드를 집어넣은 백남준의 ‘오페라 섹스트로니크’에서 ...,512
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,나아가 1984년 1월 1일 ‘굿모닝 미스터 오웰’은 세계적인 아티스트들의 퍼포먼스...,497
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,'문명의 동서남북'이라는 주제의 이 전시에서 그는 북방 유라시아의 유목 문화를 배경...,393


In [8]:
wiki_p['context_sentences'].iloc[0]

['나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.',
 '이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.',
 '# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.',
 '# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.',
 '두 목록은 모두 가나다 순이다.',
 '일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다.',
 '이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다.',
 '나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.']

In [1]:
def process_sentences(df, tokenizer, max_seq_length):
    """
    DataFrame의 각 row에 있는 문장들을 토큰화하고, 주어진 최대 시퀀스 길이에 맞게 문장을 분할하여 처리하는 함수입니다.

    각 row에 대해:
    - 각 문장을 토큰화합니다.
    - 토큰화된 문장들을 합쳐서 최대 시퀀스 길이를 넘기지 않는 범위에서 처리합니다.
    - 토큰 길이가 max_seq_length를 넘으면, 새로운 row를 생성하고 합쳐진 문장들을 저장한 후 다시 길이를 초기화합니다.
    - 모든 문장을 처리할 때까지 반복합니다.

    반환:
    - 처리된 결과가 담긴 새로운 DataFrame을 반환합니다.
    """
    new_rows = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Chunking wiki documents to fit model input size"):
        sentences = row['context_sentences']
        token_length_sum = 0
        current_text = []
        
        for sentence in sentences:
            token_length = len(tokenizer.tokenize(sentence))
            
            if token_length_sum + token_length > max_seq_length:
                new_row = row.copy()
                new_row['text_processed'] = " ".join(current_text) # text_processed 대신 text로 나중에 바꾸기
                new_rows.append(new_row)
                
                token_length_sum = 0
                current_text = []
            
            token_length_sum += token_length
            current_text.append(sentence)
        
        if current_text:
            new_row = row.copy()
            new_row['text_processed'] = " ".join(current_text)
            new_rows.append(new_row)
    
    return pd.DataFrame(new_rows)

# Process the DataFrame
processed_df = process_sentences(df=wiki_p, tokenizer=tokenizer, max_seq_length=max_seq_length)
print(processed_df.shape)
# check text_processed token length
processed_df['text_processed_token_len'] = processed_df['text_processed'].progress_apply(lambda x: len(tokenizer.tokenize(x)))
processed_df['text_processed_token_len'].describe()

NameError: name 'wiki_p' is not defined

In [53]:
# Process the DataFrame
test_df = process_sentences(df=wiki_p.iloc[:10].copy(), tokenizer=tokenizer, max_seq_length=max_seq_length)
print(test_df.shape)
# check text_processed token length
test_df['text_processed_token_len'] = test_df['text_processed'].progress_apply(lambda x: len(tokenizer.tokenize(x)))
test_df['text_processed_token_len'].describe()
test_test = test_df.iloc[:20].copy()
print(f"token length: {len(tokenizer.tokenize(wiki['text'].iloc[2]))}")
print(wiki['text'].iloc[2])
print('*'*100)
l = 0
for i in range(4, 10):
    print(f'token length: {test_test["text_processed_token_len"].iloc[i]}')
    print(test_test['text_processed'].iloc[i])
    l += test_test["text_processed_token_len"].iloc[i]
    print()
print(f'Sum of token length: {l}')

Processing Rows: 100%|██████████| 10/10 [00:00<00:00, 267.23it/s]


(17, 10)


100%|██████████| 17/17 [00:00<00:00, 848.68it/s]

token length: 2879
현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친일파인 아버지 백낙승과 어머니 조종희 사이의 3남 2녀 중 막내로 태어났다. 그후 종로구 창신동 197번지 소위 "큰대문집"에서 18세까지 살았다. 수송국민학교와 경기제1고등보통학교를 다니면서 피아니스트 신재덕에게 피아노 연주를, 작이건우]]에게 작곡을 각각 배웠다. 이때 한국이 낳은 작곡가 김순남을 사사했다. 1949년 그는 홍콩 로이덴 스쿨로 전학했으며, 한국 전쟁이 발발하기 이전 가족이 일본으로 이주했다. 그 후 일본으로 건너가 1952년 도쿄 대학교 문과부에 입학했다. 2년 후 미술사학 및 미학으로 전공을 정했지만, 실제로는 일본 당대의 작곡가 모로이 사부로, 미학자 노무라 요시오 등에게서 작곡과, 음악사학을 공부했다. 졸업 논문은 ‘아르놀트 쇤베르크 연구’이다.

1956년 백남준은 졸업과 함께 독일로 유학을 떠나 뮌헨 대학교 및 쾰른 대학교 등에서 서양의 건축, 음악사, 철학 등을 공부하였다. 뮌헨 대학교 입학 1년 후에는 프라이부르크 국립 음악 대학교로 옮겨 볼프강 포르트너 교수에게 배우지만, 곧 쇤베르크 이후 현대음악의 실험이 활발히 진행되던 다름슈타트 하기 강좌에 참여했다. 1958년 그 곳에서 현대음악가 존 케이지를 만나 그의 음악에 대한 파괴적 접근과 자유정신으로부터 깊은 영감을 얻었다. 이 영감은 "세계의 역사는 우리에게 알려준다. 주어진 게임에서 이길 수 없다면 규칙을 바꿔라" 라는 것으로 규정된다. 이후 1950년대부터 활발해지기 시작한 독일 라인 지역의 액션뮤직의 현장에서 백남준은 ‘아시아에서 온 문화테러리스트’(앨런 카프로)라고 불릴 정도의 탁월한 퍼포먼스 아티스트로 활약했다. 1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 퍼포먼스를 선보이는 것을 시작으로, 바이올린을 단숨에 파괴하거나(바이올린 솔로) 존 케이지가 착용한 넥타이를 잘라버리는 퍼포먼스(피아노 포르테를 위한 연습곡)가 

In [38]:
wiki_long = wiki_p[wiki_p['text'].apply(lambda x: sum(len(sentence) for sentence in x) > 1000)].copy()
wiki_long.head(2)

,text,corpus_source,url,domain,title,author,html,document_id,context_sentences
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...


In [42]:
test_df.head(20)

,text,corpus_source,url,domain,title,author,html,document_id,context_sentences,text_processed,text_processed_token_len
0,"나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인...",위키피디아,TODO,None,나라 목록,None,None,0,"[나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승...","나라 목록 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인...",228
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,504
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,남극 조약에 따라서 규제되는 이 권리는 어떤 나라에서도 승인을 받지 않았다. * E...,482
1,나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하...,위키피디아,TODO,None,나라 목록,None,None,1,[나라 목록 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 ...,"기사단의 단원은 모두 다른 나라의 국민이기 때문에, 사실상 이들은 모두 자신의 국가...",208
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,497
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,1959년 ‘존 케이지에게 보내는 경의’에서 음악적 콜라주와 함께 피아노를 부수는 ...,504
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,"이 전시는 백남준이 자신의 즉흥음악 또는 무음악의 발상에 기초한 실제 퍼포먼스, 그...",479
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,특히 1967년 음악에 성적인 코드를 집어넣은 백남준의 ‘오페라 섹스트로니크’에서 ...,512
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,나아가 1984년 1월 1일 ‘굿모닝 미스터 오웰’은 세계적인 아티스트들의 퍼포먼스...,497
2,백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이...,위키피디아,TODO,None,백남준,None,None,2,[백남준 현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신...,'문명의 동서남북'이라는 주제의 이 전시에서 그는 북방 유라시아의 유목 문화를 배경...,393
